In [161]:
import requests
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import json
from os import path

# Парсинг
### Создание функции для парсинга
В ходе исследования возможностей для сбора данных, один из наборов данных был проверен вручную и были выявлены нужные для нашей задачи столбцы, остальные были отброшены. Так же для удобства были подсчитаны некоторые новые столбцы. Всё имеет говорящее название, смотрите код ниже

In [162]:
headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }
datetime_columns = ['DATETIME_DEPARTURE_PLAN_MSK', 'DATETIME_DEPARTURE_ACTUAL_MSK', 'DATETIME_ARRIVAL_PLAN_MSK', 'DATETIME_ARRIVAL_ACTUAL_MSK']

def create_df(station_name: str, url: str, day: str) -> pd.DataFrame:
    '''url можно получить способом упомянутым в отчёте; day вводить в формате дд.мм.гггг'''
    response = requests.get(url + day, headers=headers)
    json_ = response.json()

    if json_.get('TABLO') is None or json_['TABLO'].get('TABLO_TRAIN') is None:
        print(f"Нет данных для {station_name} за {day}")
        return
    
    with open(f'data/json/{station_name}-{day}.json', 'w') as f:
        json.dump(json_['TABLO']['TABLO_TRAIN'], f)
        
    df = pd.read_json(f'data/json/{station_name}-{day}.json')

    df = df[['TRAIN_NAME', *datetime_columns]]
    df[datetime_columns] = df[datetime_columns].apply(pd.to_datetime)

    df['DATETIME_ARRIVAL_DELAY_MSK'] = df['DATETIME_ARRIVAL_ACTUAL_MSK'] - df['DATETIME_ARRIVAL_PLAN_MSK']
    df['DATETIME_DEPARTURE_DELAY_MSK'] = df['DATETIME_DEPARTURE_ACTUAL_MSK'] - df['DATETIME_DEPARTURE_PLAN_MSK']
    df['day'] = day
    return df

### Подготовка параметров парсинга

In [163]:
# Файл urls.json был собран вручную
with open('urls.json') as f:
    urls = json.load(f)
# Для тестов была выбрана одна неделя из периода проведения практики
days = ['08.07.2024', '09.07.2024', '10.07.2024', '11.07.2024', '12.07.2024', '13.07.2024', '14.07.2024']

### Непосредственно парсинг и запись данных в файлы

In [164]:
for station_name, url in urls.items():
    dfs = []
    for day in days:
        dfs.append(create_df(station_name, url, day))
    for i in dfs:
        if i is not None: 
            pd.concat(dfs).to_csv(f'data/csv/{station_name}.csv')
            break
    else:
        print(f'Для {station_name} не найдено данных, файл не записан')

Нет данных для Вокзал Восточный (терминал Черкизово) за 08.07.2024
Нет данных для Вокзал Восточный (терминал Черкизово) за 09.07.2024
Нет данных для Вокзал Восточный (терминал Черкизово) за 10.07.2024
Нет данных для Вокзал Восточный (терминал Черкизово) за 11.07.2024
Нет данных для Вокзал Восточный (терминал Черкизово) за 12.07.2024
Нет данных для Вокзал Восточный (терминал Черкизово) за 13.07.2024
Нет данных для Вокзал Восточный (терминал Черкизово) за 14.07.2024
Для Вокзал Восточный (терминал Черкизово) не найдено данных, файл не записан


# Анализ данных и построение графиков
### Считывание всех данных из файлов и приведение их к нужным типам данных

In [247]:
dfs = {}
for station in urls.keys():
    if path.isfile(f'data/csv/{station}.csv'):
        df = pd.read_csv(f'data/csv/{station}.csv').drop(columns=['Unnamed: 0'])
        df[datetime_columns] = df[datetime_columns].apply(pd.to_datetime)
        df['DATETIME_ARRIVAL_DELAY_MSK'] = pd.to_timedelta(df['DATETIME_ARRIVAL_DELAY_MSK'])
        df['DATETIME_DEPARTURE_DELAY_MSK'] = pd.to_timedelta(df['DATETIME_DEPARTURE_DELAY_MSK'])
        dfs[station] = df

In [248]:
dfs.keys() # Смотрим итоговый список вокзалов

dict_keys(['Белорусский вокзал', 'Вокзал Царицыно', 'Казанский вокзал', 'Киевский вокзал', 'Курский вокзал', 'Ленинградский вокзал', 'Павелецкий вокзал', 'Савеловский вокзал', 'Ярославский вокзал'])

In [249]:
dfs['Белорусский вокзал'].head()

,TRAIN_NAME,DATETIME_DEPARTURE_PLAN_MSK,DATETIME_DEPARTURE_ACTUAL_MSK,DATETIME_ARRIVAL_PLAN_MSK,DATETIME_ARRIVAL_ACTUAL_MSK,DATETIME_ARRIVAL_DELAY_MSK,DATETIME_DEPARTURE_DELAY_MSK,day
0,6320,2024-07-08 00:01:00,NaT,2024-07-08 00:00:00,NaT,NaT,NaT,08.07.2024
1,6886,2024-07-08 00:01:00,2024-07-08 00:06:00,2024-07-08 00:00:00,2024-07-08 00:04:00,0 days 00:04:00,0 days 00:05:00,08.07.2024
2,7376,2024-07-08 00:05:00,NaT,2024-07-08 00:05:00,2024-07-08 00:07:00,0 days 00:02:00,NaT,08.07.2024
3,6075,2024-07-08 00:06:00,NaT,2024-07-08 00:06:00,2024-07-08 00:06:00,0 days 00:00:00,NaT,08.07.2024
4,6076,2024-07-08 00:06:00,2024-07-08 00:09:00,2024-07-08 00:06:00,NaT,NaT,0 days 00:03:00,08.07.2024


In [250]:
dfs['Белорусский вокзал'].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5152 entries, 0 to 5151
Data columns (total 8 columns):
 #   Column                         Non-Null Count  Dtype          
---  ------                         --------------  -----          
 0   TRAIN_NAME                     5152 non-null   int64          
 1   DATETIME_DEPARTURE_PLAN_MSK    5152 non-null   datetime64[ns] 
 2   DATETIME_DEPARTURE_ACTUAL_MSK  2251 non-null   datetime64[ns] 
 3   DATETIME_ARRIVAL_PLAN_MSK      5152 non-null   datetime64[ns] 
 4   DATETIME_ARRIVAL_ACTUAL_MSK    2270 non-null   datetime64[ns] 
 5   DATETIME_ARRIVAL_DELAY_MSK     2270 non-null   timedelta64[ns]
 6   DATETIME_DEPARTURE_DELAY_MSK   2251 non-null   timedelta64[ns]
 7   day                            5152 non-null   object         
dtypes: datetime64[ns](4), int64(1), object(1), timedelta64[ns](2)
memory usage: 322.1+ KB


In [251]:
dfs['Белорусский вокзал'].isnull().sum() / len(dfs['Белорусский вокзал']) * 100

TRAIN_NAME                        0.000000
DATETIME_DEPARTURE_PLAN_MSK       0.000000
DATETIME_DEPARTURE_ACTUAL_MSK    56.308230
DATETIME_ARRIVAL_PLAN_MSK         0.000000
DATETIME_ARRIVAL_ACTUAL_MSK      55.939441
DATETIME_ARRIVAL_DELAY_MSK       55.939441
DATETIME_DEPARTURE_DELAY_MSK     56.308230
day                               0.000000
dtype: float64

Пропусков более 50%, а с учётом того, что они в полях с фактическим временем прибытия и отправки, это явная проблема. Появляется вопрос: являются ли пропуски отсутствием рейса в тот день или просто фактическое время не было записано? 
Я считаю, что первый вариант верен и просто поездка не состоялась => удаляем строки, где пропущено и время прибытия и время отправления

In [252]:
for df in dfs.values():
    df.dropna(thresh=6, inplace=True)

In [253]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2867 entries, 0 to 3623
Data columns (total 8 columns):
 #   Column                         Non-Null Count  Dtype          
---  ------                         --------------  -----          
 0   TRAIN_NAME                     2867 non-null   int64          
 1   DATETIME_DEPARTURE_PLAN_MSK    2867 non-null   datetime64[ns] 
 2   DATETIME_DEPARTURE_ACTUAL_MSK  1429 non-null   datetime64[ns] 
 3   DATETIME_ARRIVAL_PLAN_MSK      2867 non-null   datetime64[ns] 
 4   DATETIME_ARRIVAL_ACTUAL_MSK    1438 non-null   datetime64[ns] 
 5   DATETIME_ARRIVAL_DELAY_MSK     1438 non-null   timedelta64[ns]
 6   DATETIME_DEPARTURE_DELAY_MSK   1429 non-null   timedelta64[ns]
 7   day                            2867 non-null   object         
dtypes: datetime64[ns](4), int64(1), object(1), timedelta64[ns](2)
memory usage: 201.6+ KB


### Построение графиков

In [254]:
daily_load_data = []
for station, df in dfs.items():
    daily_load = df.groupby('day').size().reset_index(name='count')
    daily_load['Вокзал'] = station
    daily_load_data.append(daily_load)
daily_load_df = pd.concat(daily_load_data)

avg_load = daily_load_df.groupby('Вокзал')['count'].mean().sort_values(ascending=False).reset_index()
daily_load_df = daily_load_df.merge(avg_load, on='Вокзал', suffixes=('', '_avg'))


fig = px.line(daily_load_df, x='day', y='count', color='Вокзал', title='Загруженность вокзалов по дням', labels={'count':'Количество поездов', 'day_label':'Дата'}, category_orders={'Вокзал': avg_load['Вокзал'].tolist()})
fig.update_layout(xaxis_tickangle=-45, template='plotly_dark')
fig.show()

С данными лишь за одну неделю сложно делать какие-то конкретные выводы, но даже здесь заметно, что в выходные у всех вокзалов меньше нагрузка. Вероятно это связано с тем, что в выходные многие люди не пользуются пригородными электричками, для поездки из области в Москву на работу или учёбу.

In [255]:
punctuality_data = []
delay_strength_data = []

for station, df in dfs.items():
    departure_ontime = df['DATETIME_DEPARTURE_DELAY_MSK'].dropna().apply(lambda x: x.total_seconds() <= 0).mean() * 100
    punctuality_data.append({'Вокзал': station, 'Процент отправлений вовремя': departure_ontime})
    
    df['positive_delay'] = df['DATETIME_DEPARTURE_DELAY_MSK'].apply(lambda x: max(pd.to_timedelta(0), x) if pd.notnull(x) else pd.to_timedelta(0))
    total_delay_seconds = df['positive_delay'].apply(lambda x: x.total_seconds()).sum()
    num_flights = len(df)
    avg_delay_strength = total_delay_seconds / num_flights if num_flights > 0 else 0
    delay_strength_data.append({'Вокзал': station, 'Среднее опоздание (сек/рейс)': avg_delay_strength})

punctuality_df = pd.DataFrame(punctuality_data)
delay_strength_df = pd.DataFrame(delay_strength_data)


punctuality_df = punctuality_df.sort_values(by='Процент отправлений вовремя', ascending=False)
delay_strength_df['Вокзал'] = pd.Categorical(delay_strength_df['Вокзал'], categories=punctuality_df['Вокзал'], ordered=True)
delay_strength_df = delay_strength_df.sort_values(by='Вокзал')


punctuality_df['color'] = 'blue'
max_punctuality_index = punctuality_df['Процент отправлений вовремя'].idxmax()
min_punctuality_index = punctuality_df['Процент отправлений вовремя'].idxmin()
punctuality_df.at[max_punctuality_index, 'color'] = 'green'
punctuality_df.at[min_punctuality_index, 'color'] = 'red'

delay_strength_df['color'] = 'blue'
max_delay_strength_index = delay_strength_df['Среднее опоздание (сек/рейс)'].idxmin()
min_delay_strength_index = delay_strength_df['Среднее опоздание (сек/рейс)'].idxmax()
delay_strength_df.at[max_delay_strength_index, 'color'] = 'green'
delay_strength_df.at[min_delay_strength_index, 'color'] = 'red'


fig = make_subplots(rows=2, cols=1, shared_xaxes=True, subplot_titles=('Пунктуальность отправлений', 'Среднее опоздание'), vertical_spacing=0.15)

fig.add_trace(
    go.Bar(x=punctuality_df['Вокзал'], y=punctuality_df['Процент отправлений вовремя'], marker_color=punctuality_df['color'], name='Процент отправлений вовремя'),
    row=1, col=1
)
fig.add_trace(
    go.Bar(x=delay_strength_df['Вокзал'], y=delay_strength_df['Среднее опоздание (сек/рейс)'], marker_color=delay_strength_df['color'], name='Среднее опоздание (сек/рейс)'),
    row=2, col=1
)

fig.update_layout(title_text='Пунктуальность и среднее опоздание по вокзалам', showlegend=False, height=700, template='plotly_dark')
fig.update_xaxes(title_text='Вокзал', row=2, col=1)
fig.update_yaxes(title_text='Процент отправлений вовремя', row=1, col=1)
fig.update_yaxes(title_text='Среднее опоздание (сек/рейс)', row=2, col=1)

fig.show()

Ленинградский вокзал демонстрирует невероятные пунктуальность, тем не менее, если взять в расчёт среднюю загруженость вокзалов, то становится ясно, что она сильно влияет на итоговые показатели опозданий, что совершенно логично.

In [256]:
selected_station = 'Белорусский вокзал'
df = dfs[selected_station].copy()
df['day'] = pd.to_datetime(df['DATETIME_DEPARTURE_PLAN_MSK']).dt.date


daily_load = df.groupby('day').size().reset_index(name='count')

df['on_time_departure'] = df['DATETIME_DEPARTURE_DELAY_MSK'].dropna().apply(lambda x: x.total_seconds() <= 0)
punctuality_by_day = df.groupby('day')['on_time_departure'].mean().reset_index(name='Процент отправлений вовремя')
punctuality_by_day['Процент отправлений вовремя'] *= 100

df['positive_delay'] = df['DATETIME_DEPARTURE_DELAY_MSK'].apply(lambda x: max(pd.to_timedelta(0), x) if pd.notnull(x) else pd.to_timedelta(0))
delay_by_day = df.groupby('day')['positive_delay'].apply(lambda x: x.apply(lambda y: y.total_seconds()).mean()).reset_index(name='Среднее опоздание (сек/рейс)')


punctuality_by_day['color'] = 'blue'
max_punctuality_index = punctuality_by_day['Процент отправлений вовремя'].idxmax()
min_punctuality_index = punctuality_by_day['Процент отправлений вовремя'].idxmin()
punctuality_by_day.at[max_punctuality_index, 'color'] = 'green'
punctuality_by_day.at[min_punctuality_index, 'color'] = 'red'

delay_by_day['color'] = 'blue'
max_delay_index = delay_by_day['Среднее опоздание (сек/рейс)'].idxmin()
min_delay_index = delay_by_day['Среднее опоздание (сек/рейс)'].idxmax()
delay_by_day.at[max_delay_index, 'color'] = 'green'
delay_by_day.at[min_delay_index, 'color'] = 'red'


fig = make_subplots(rows=3, cols=1, shared_xaxes=True, subplot_titles=('Загруженность по дням', 'Процент пунктуальности по дням', 'Среднее опоздание по дням'), vertical_spacing=0.15)

fig.add_trace(
    go.Scatter(x=daily_load['day'], y=daily_load['count'], mode='lines+markers', name='Загруженность'),
    row=1, col=1
)
fig.add_trace(
    go.Bar(x=punctuality_by_day['day'], y=punctuality_by_day['Процент отправлений вовремя'], marker_color=punctuality_by_day['color'], name='Процент отправлений вовремя'),
    row=2, col=1
)
fig.add_trace(
    go.Bar(x=delay_by_day['day'], y=delay_by_day['Среднее опоздание (сек/рейс)'], marker_color=delay_by_day['color'], name='Среднее опоздание (сек/рейс)'),
    row=3, col=1
)


fig.update_layout(title_text=f'Анализ: {selected_station}', showlegend=False, height=900, template='plotly_dark')
fig.update_xaxes(title_text='Дата', row=3, col=1)
fig.update_yaxes(title_text='Количество поездов', row=1, col=1)
fig.update_yaxes(title_text='Процент отправлений вовремя', row=2, col=1)
fig.update_yaxes(title_text='Среднее опоздание (сек/рейс)', row=3, col=1)

fig.show()

Возвращаясь к прошлому выводу - самый пунктуальный день у вокзала выпал на день, с неожиданно нихкой нагрузкой.

In [261]:
selected_stations = ['Белорусский вокзал', 'Вокзал Царицыно']
colors = dict(zip(selected_stations, ['orange', 'turquoise']))


data_frames = {station: dfs[station].copy() for station in selected_stations}
for df in data_frames.values():
    df['day'] = pd.to_datetime(df['DATETIME_DEPARTURE_PLAN_MSK']).dt.date


daily_load = {station: df.groupby('day').size().reset_index(name='count') for station, df in data_frames.items()}

for df in data_frames.values():
    df['on_time_departure'] = df['DATETIME_DEPARTURE_DELAY_MSK'].dropna().apply(lambda x: x.total_seconds() <= 0)
punctuality_by_day = {station: df.groupby('day')['on_time_departure'].mean().reset_index(name='Процент отправлений вовремя') for station, df in data_frames.items()}
for df in punctuality_by_day.values():
    df['Процент отправлений вовремя'] *= 100

for df in data_frames.values():
    df['positive_delay'] = df['DATETIME_DEPARTURE_DELAY_MSK'].apply(lambda x: max(pd.to_timedelta(0), x) if pd.notnull(x) else pd.to_timedelta(0))
delay_by_day = {station: df.groupby('day')['positive_delay'].apply(lambda x: x.apply(lambda y: y.total_seconds()).mean()).reset_index(name='Среднее опоздание (сек/рейс)') for station, df in data_frames.items()}


fig = make_subplots(rows=3, cols=1, shared_xaxes=True, subplot_titles=('Загруженность по дням', 'Процент пунктуальности по дням', 'Среднее опоздание по дням'), vertical_spacing=0.15)

for station in selected_stations:
    fig.add_trace(
        go.Scatter(x=daily_load[station]['day'], y=daily_load[station]['count'], mode='lines+markers', name=f'Загруженность {station}', line=dict(color=colors[station])),
        row=1, col=1
    )
for station in selected_stations:
    fig.add_trace(
        go.Bar(x=punctuality_by_day[station]['day'], y=punctuality_by_day[station]['Процент отправлений вовремя'], name=f'Процент пунктуальности {station}', marker_color=colors[station]),
        row=2, col=1
    )
for station in selected_stations:
    fig.add_trace(
        go.Bar(x=delay_by_day[station]['day'], y=delay_by_day[station]['Среднее опоздание (сек/рейс)'], name=f'Среднее опоздание {station}', marker_color=colors[station]),
        row=3, col=1
    )


fig.update_layout(title_text=f'Сравнительный анализ: <span style="color:{colors[selected_stations[0]]};">{selected_stations[0]}</span> и <span style="color:{colors[selected_stations[1]]};">{selected_stations[1]}</span>', showlegend=False, height=900, template='plotly_dark')
fig.update_xaxes(title_text='Дата', row=3, col=1)
fig.update_yaxes(title_text='Количество поездов', row=1, col=1)
fig.update_yaxes(title_text='Процент отправлений вовремя', row=2, col=1)
fig.update_yaxes(title_text='Среднее опоздание (сек/рейс)', row=3, col=1)

fig.show()

Этот график демонстрирует возможность удобного сравнения двух вокзалов, выбраных по названиям.